In [1]:
from igraph import *
import xnet
import glob
import math
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
from collections import defaultdict
from scipy.linalg import norm

In [2]:
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))

matplotlib: 3.0.2


In [3]:
%load_ext autoreload
%autoreload

# Leitura dos dados
calculate_dist é chamado apenas uma vez e depois a informação é salva na própria rede que passa a ser lida

In [4]:
def norm(X):
    minx = min(X)
    maxx = max(X)
    X = (np.asarray(X)-minx)/(maxx-minx)
    return X

def calculate_dist(filenames):
	for filename in filenames:
		# print(filename)
		net = xnet.xnet2igraph(filename)
		weights = net.es['weight']
		weights = norm(weights)
		weights = [math.sqrt(2*(1-w)) for w in weights]
		print(weights[:10])
		if len(weights) > 0:
			net.es['distance'] = weights
			xnet.igraph2xnet(net,filename[:-5]+"_dist.xnet")
		else:
			print('error',filename)

In [5]:
# Executa apenas uma vez
# filenames = glob.glob('colabs/basic_colab_cut/*pac*.xnet')
# calculate_dist(filenames)

In [4]:
filenames = glob.glob('colabs/wbasic/*pac*5.xnet')
filenames = sorted(filenames)

In [6]:
filenames = glob.glob('colabs/basic_colab_cut/*pac*dist.xnet')
filenames = sorted(filenames)
print(filenames)

['colabs/basic_colab_cut/colab_1990_1993.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1991_1994.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1992_1995.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1993_1996.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1994_1997.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1995_1998.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1996_1999.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1997_2000.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1998_2001.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_1999_2002.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_2000_2003.xnet_0.5_selected_basic4_pac_labels_dist.xnet', 'colabs/basic_colab_cut/colab_2

In [7]:
def calculate_shortest_paths(net):
    summary = defaultdict(lambda:defaultdict(lambda:0))
    all_paths = []
    
    pacs = [str(i) for i in range(10)]
    
    for pac1 in pacs:
        print(pac1)
        sources = net.vs.select(pac_label_eq=pac1)[:10]
        for pac2 in pacs:
            targets = net.vs.select(pac_label_eq=pac2)[:10]
            targets = [v.index for v in targets]
            paths = []
            for s in sources:
                path_lens = net.get_shortest_paths(s,to=targets,weights='distance',output="epath")
                for p in path_lens:
                    x = sum(net.es[idx]['distance'] for idx in p)
#                     print(x,end=',')
                    if x > 0:
                        paths.append(x)
                        all_paths.append(x)
            if len(paths) == 0:
                print('len path == 0',len(targets),len(sources))
                summary[pac1][pac2] = (np.nan,np.nan,np.nan)
                summary[pac2][pac1] = (np.nan,np.nan,np.nan)
            else:
                mean = np.mean(paths)
                std_dev = np.std(paths)
                summary[pac1][pac2] = (mean,std_dev,len(targets)) 
                summary[pac2][pac1] = (mean,std_dev,len(sources))

            if pac1 == pac2:
                break
    all_paths_mean = np.mean(all_paths)
    all_paths_std = np.std(all_paths)
    return summary,(all_paths_mean,all_paths_std)

In [8]:
def plot(summaries,all_paths_summary,dates,base=''):
    # xs = [float(d) for d in dates]
    ys = defaultdict(lambda:defaultdict(lambda:[]))

    for d in dates:
        year_summary = summaries[d]
        for pp1,summary_pp1 in year_summary.items():
            if summary_pp1:
                for pp2,(mean,std,f) in summary_pp1.items():
                    ys[pp1][pp2].append((d,mean,std,f))
    
    for pp1,pp2_means in ys.items():
        freq = []
        for pp2,means_std in pp2_means.items():
            means_std = np.array(means_std)
            freq.append((np.nanmean(means_std[:,3]),pp2))
        freq = sorted(freq,reverse=True)
        f_max = freq[0][0]
        plt.figure(figsize=(12,3))

        for f,pp2 in freq:
            means_std = pp2_means[pp2]
            means_std = np.asarray(means_std)
            means = means_std[:,1]
            std = means_std[:,2]
            xs = means_std[:,0]
#             print(xs)
            
            fraq = f/f_max
            elw = max(0.3,2*fraq)
            lw = max(0.3,2*fraq)
            ms = max(0.3,2*fraq)
            
            alpha = max(0.3,0.65*fraq)
            
            plt.errorbar(xs,means,std,
            linestyle='-',label=pp2.upper(),fmt='o',elinewidth=elw,linewidth=lw,markersize=ms,alpha=alpha)#,color=pps_color[pp2])
        all_paths_summary = np.asarray(all_paths_summary)
        
        labels = [str(int(x)) if x%5 == 0 else '' for x in xs]
        plt.xticks(np.arange(min(xs), max(xs)+1, 1.0),labels=labels)   
        
        plt.errorbar(xs, all_paths_summary[:,0], all_paths_summary[:,1], 
        linestyle='-',label='all',fmt='o',elinewidth=1,linewidth=1.2,markersize=1.5,alpha=0.3,color='cyan')
        plt.legend(loc='upper right')
        plt.xlabel('year')
        plt.ylabel('shortest path mean')                
        plt.savefig('shortest_paths_'+base+pp1+'.pdf',format='pdf',bbox_inches="tight")
        plt.clf()

In [6]:
dates = []
nets = []
for filename in filenames:
    net = xnet.xnet2igraph(filename)
    nets.append(net.components().giant())

    # por ano
    date = int(filename.split('colab_')[1].split('_')[0])
    
    dates.append(date)
print(dates)

[1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005]


In [7]:
for y,n in zip(dates,nets):
    print(y)
    print(np.unique(n.vs['pac_label'],return_counts=True))

1990
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([12655,  8894,    82,   418,   358]))
1991
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([19556,   523,   145,     7,  4302]))
1992
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([19160,  2302,  5255,   191,   105]))
1993
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([20057,  2660,   166,   162,  5477]))
1994
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([21845,  2762,   334,   104,  6390]))
1995
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([17454,  3052,   236,  5081,  6734]))
1996
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([ 3013, 24698,   586,  5957,  1189]))
1997
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([  938, 26072,    18,  6924,  3627]))
1998
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([ 3637, 18925,   833,  8867,  7050]))
1999
(array(['0', '1', '2', '3', '4'], dtype='<U1'), array([32432,  1831,  5663,    13,    31]))
2000
(array(['0', '1', '2', '3

In [28]:
dates = np.asarray(dates)
nets = np.asarray(nets)

sorted_idxs = np.argsort(dates)
dates = dates[sorted_idxs]
nets = nets[sorted_idxs]

Código a seguir válido para quando as redes são geradas para cada mês com janela de 3 meses:

Código válido apenas para intervalo de tempo em anos:

In [59]:
summaries = dict()
all_paths_summary = []
for date,net in zip(dates,nets):
    summaries[date],all_paths = calculate_shortest_paths(net)
    all_paths_summary.append(all_paths)

0
1
len path == 0 1 1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
len path == 0 10 0
len path == 0 10 0
len path == 0 10 0
len path == 0 10 0
len path == 0 3 0
len path == 0 0 0
6
len path == 0 0 2
7
len path == 0 0 10
8
len path == 0 0 10
9
len path == 0 0 10
0
1
2
3
4
5
len path == 0 1 1
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
len path == 0 10 0
len path == 0 10 0
len path == 0 10 0
len path == 0 4 0
len path == 0 3 0
len path == 0 10 0
len path == 0 3 0
len path == 0 0 0
8
len path == 0 0 3
9
len path == 0 0 1
len path == 0 1 1
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
len path == 0 3 0
len path == 0 10 0
len path == 0 10 0
len path == 0 10 0
len path == 0 10 0
len path == 0 10 0
len path == 0 0 0
7
len path == 0 0 2
8
len path == 0 0 3
9
len path == 0 0 4
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
len path == 0 1 1
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
len path == 0 1 1
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6


In [60]:
plot(summaries,all_paths_summary,dates)

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

# Isolamento

In [61]:
def plot_isolation(summaries,all_paths_summary,dates,base='',color=None):
    isolation = dict()
    # xs = [float(d) for d in dates]
    ys = defaultdict(lambda:defaultdict(lambda:[]))

    freq_dict = defaultdict(lambda:[])
    for d in dates:
        year_summary = summaries[d]
        for pp1,summary_pp1 in year_summary.items():
            if summary_pp1:
                for pp2,(mean,std,f) in summary_pp1.items():
                    ys[pp1][pp2].append((d,mean,std,f))
                    freq_dict[pp2].append(f)

    freq = [(np.mean(fs),pp) for pp,fs in freq_dict.items()]
    freq = sorted(freq,reverse=True)
    f_max = freq[0][0]
    if np.isnan(f_max):
        return None,None
    plt.figure(figsize=(12,3))
                    
    for f,pp1 in freq:
        pp2_means = ys[pp1]
        
        means = np.zeros(len(pp2_means[pp1]))
        total_std = np.zeros(len(pp2_means[pp1]))
        total = np.zeros(len(pp2_means[pp1]))
        xs = []
        for pp2,means_std in pp2_means.items():
            if not pp1 == pp2:
                means_std = np.array(means_std)
                xs = means_std[:,0]
                t = means_std[:,3]
                std = means_std[:,2]
                print(t.shape,total.shape)
                total += t
#                 print(means_std[:,1],t,means_std[:,1]*t)
                means += means_std[:,1]*t
                total_std += std*t

        means /= total
        total_std /= total
        fraq = f/f_max
        fraq = max(fraq,0.4)
        plt.errorbar(xs,means,total_std,
        linestyle='-',label=pp1.upper(),fmt='o',elinewidth=1.5*fraq,linewidth=2*fraq,markersize=2*fraq,alpha=0.65*fraq)#,color=color[pp1])
        
        isolation[pp1] = (means,total_std)
    
#     print(xs)
#     labels = [str(int(x))  if x%5 == 0 else '' for x in xs]  
#     print(labels)
#     plt.xticks(np.arange(min(xs), max(xs)+1, 1.0),labels=labels)   
    
#     for begin,delta,color in interval_colors:
#         if begin+delta >= dates[0] and begin <= dates[-1]:
#             plt.axvspan(max(begin,dates[0]), min(begin+delta,dates[-1]), facecolor=color, alpha=0.3)

    plt.legend(loc='upper right')
    plt.xlabel('year')
    plt.ylabel('isolation')                
    plt.savefig('shortest_paths_'+base+'.pdf',format='pdf',bbox_inches="tight")
    plt.clf()
    return isolation,xs

Código para dados em intervalos de anos:

In [62]:
isolation,xs = plot_isolation(summaries,all_paths_summary,dates)

# Fragmentação

In [51]:
def plot_fragmentation(summaries,all_paths_summary,pps,dates,inverval_colors,color,base):
    # xs = [float(d) for d in dates]
    ys = defaultdict(lambda:defaultdict(lambda:[]))

    freq_dict = defaultdict(lambda:[])
    for d in dates:
        year_summary = summaries[d]
        for pp1,summary_pp1 in year_summary.items():
            if summary_pp1:
                for pp2,(mean,std,f) in summary_pp1.items():
                    if pp1 == pp2:
                        ys[pp1][pp2].append((d,mean,std,f))
                        freq_dict[pp2].append(f)

    freq = [(np.mean(fs),pp) for pp,fs in freq_dict.items()]
    freq = sorted(freq,reverse=True)
    f_max = freq[0][0]
    plt.figure(figsize=(12,3))
    
    fragmentation = dict()
    for f,pp1 in freq:
        pp2_means = ys[pp1]
        
        means = np.zeros(len(pp2_means[pp1]))
        xs = []
        for pp2,means_std in pp2_means.items():
            if pp1 == pp2:
                means_std = np.array(means_std)
                means = means_std[:,1]
                std = means_std[:,2]
                xs = means_std[:,0]
                break
        fraq = f/f_max
        fraq = max(fraq,0.45)
        if np.isnan(fraq):
            print(pp1,f,f_max)
            continue
        plt.subplot(1,1,1)
        plt.errorbar(xs,means,std,
        linestyle='-',label=pp1.upper(),fmt='o',elinewidth=1.5*fraq,linewidth=2*fraq,markersize=2*fraq,alpha=0.65*fraq)#,color=color[pp1])
        fragmentation[pp1] = (means,std)
        
#     labels = [str(int(x))  if x%5 == 0 else '' for x in xs]
#     plt.xticks(np.arange(min(xs), max(xs)+1, 1.0),labels=labels)   
    
    for begin,delta,color in interval_colors:
        if begin+delta >= dates[0] and begin <= dates[-1]:
            plt.axvspan(max(begin,dates[0]), min(begin+delta,dates[-1]), facecolor=color, alpha=0.3)

    
    plt.legend(loc='upper right')
    plt.xlabel('year')
    plt.ylabel('fragmentation')                
    plt.savefig('shortest_paths_'+base+'.pdf',format='pdf',bbox_inches="tight")
    
    plt.clf()
    return fragmentation,xs

In [21]:
def plot_4years_fragmentation(dates,nets,year,valid_pps):
    summaries = dict()

    idxs = [idx for idx,date in enumerate(dates) if date < year+4 and date >= year]
#   
    current_dates = []
    current_nets = []
    all_paths_summary = []

    current_nets = [nets[idx] for idx in idxs]
#     valid_pps = list(get_valid_pp(current_nets))
    valid_pps = list(get_valid_pp(current_nets,cut_percent=0.05))
#     valid_pps = set()
#     for n in current_nets:
#         valid_pps |= set(n.vs['political_party'])
#     print(valid_pps)

    for idx in idxs:
        current_dates.append(dates[idx])
        summaries[dates[idx]],all_paths = calculate_shortest_paths(nets[idx],valid_pps)
        all_paths_summary.append(all_paths)
        
    return plot_fragmentation(summaries,all_paths_summary,valid_pps,current_dates,interval_colors,pps_color,str(year)+'_'+str(year+3)+'_fragmentation')

Código para intervalo em anos:

In [28]:
frag,xs = plot_4years_fragmentation(dates,nets,1995,valid_pps)

<Figure size 864x216 with 0 Axes>

Código para intervalo em meses:

In [53]:
total_frag = defaultdict(lambda:[])
total_xs = []
for year in range(1995,2018,4):
    frag,xs = plot_4years_fragmentation(dates,nets,year,valid_pps)
    for k,v in frag.items():
        total_frag[k].append(v)
    total_xs.append(xs)

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

<Figure size 864x216 with 0 Axes>

Distância em relação ao governo:

In [76]:
intervals = [('FHC',(1995.01,2003)),('Lula',(2003.01,2011)),('Dilma',(2011.01,2016.4)),('Temer',(2016.66,2019))]
gov_map = {'FHC':'psdb','Lula':'pt','Dilma':'pt','Temer':'pmdb'}

def calculate_align(summaries,intervals,dates,pp):
    align = defaultdict(lambda:[])
    for d in dates:
        gov = None
        for president,interval in intervals:
            if d >= interval[0] and d <= interval[1]:
                gov = gov_map[president]
                break
        summary = summaries[d][pp][gov]
        
        if not summary == 0:
            mean = summary[0]
            align[president].append(mean)
    
    align_mean = [(k,np.mean(a)) for k,a in align.items()]
    return align_mean

def aligns(dates,nets,valid_pps):
    summaries = dict()

    for date,net in zip(dates,nets):
        summaries[date],_ = calculate_shortest_paths(net,valid_pps)
    
    for pp in valid_pps:
        align_mean = calculate_align(summaries,intervals,dates,pp)
        print(pp,align_mean)

aligns(dates,nets,valid_pps)

psdb [('FHC', 2.4908820766416526), ('Lula', 3.8835780174612617), ('Dilma', 4.095665412007527), ('Temer', 2.8818087798223324)]
pp [('FHC', 2.7917070797067045), ('Lula', 2.7688764509675385), ('Dilma', 3.666234855916504), ('Temer', 2.821914764948892)]
pmdb [('FHC', 2.8051044997743655), ('Lula', 2.780354046114604), ('Dilma', 3.5343510334110326), ('Temer', 2.6642746684162524)]
pt [('FHC', 3.8751642380542393), ('Lula', 2.0597650478176948), ('Dilma', 1.9066299336191868), ('Temer', 4.386559097832085)]
dem [('FHC', 2.6182757068557603), ('Lula', 3.8935104225662798), ('Dilma', 4.201142880993076), ('Temer', 2.727031004680421)]


# Medida de isolamento/fragmentação médios por intervalo

In [58]:
def metric_mean(pp,intervals,total_isolation,total_xs,metric):
    isolation_pp = total_isolation[pp]
    gov_iso,gov_count,op_iso,op_count = 0.0,0,0.0,0
    value_by_interval = [[] for _ in range(len(intervals))]
    for xs,iso in zip(total_xs,isolation_pp):
        for idx,x in enumerate(xs):
            for i in range(len(intervals)):
                
                if x >= intervals[i][0] and x <= intervals[i][1]:
                    value_by_interval[i].append(iso[0][idx])
#                     print(value_by_interval)
    for i,value in zip(intervals,value_by_interval):
        print(metric+' of '+pp+' during '+str(i),mean(value))

In [59]:
intervals = [(1995.01,2003),(2003.01,2011),(2011.01,2016.4),(2016.66,2019)]
for pp in valid_pps:
    metric_mean(pp,intervals,total_isolation,total_xs,'Isolation')
    print()

Isolation of psdb during (1995.01, 2003) 2.937746518528365
Isolation of psdb during (2003.01, 2011) 3.4379430689604016
Isolation of psdb during (2011.01, 2016.4) 3.622686649126459
Isolation of psdb during (2016.66, 2019) 3.3358949149289736

Isolation of pp during (1995.01, 2003) 2.9595197386087437
Isolation of pp during (2003.01, 2011) 3.1006175699889273
Isolation of pp during (2011.01, 2016.4) 3.431763452369908
Isolation of pp during (2016.66, 2019) 3.338128443687539

Isolation of pmdb during (1995.01, 2003) 2.956948742582771
Isolation of pmdb during (2003.01, 2011) 3.187213250056103
Isolation of pmdb during (2011.01, 2016.4) 3.4705848806008075
Isolation of pmdb during (2016.66, 2019) 3.283016377881835

Isolation of pt during (1995.01, 2003) 3.793930002186151
Isolation of pt during (2003.01, 2011) 3.3269296511864677
Isolation of pt during (2011.01, 2016.4) 3.795111096825172
Isolation of pt during (2016.66, 2019) 4.371097939688608

Isolation of dem during (1995.01, 2003) 2.953175946486

In [91]:
for pp in valid_pps:
    metric_mean(pp,intervals,total_frag,total_xs,'Fragmentation')
    print()

Fragmentation of psdb during (1995.01, 2003) 2.4908820766416517
Fragmentation of psdb during (2003.01, 2011) 2.0328737211304846
Fragmentation of psdb during (2011.01, 2016.4) 1.988783093836204
Fragmentation of psdb during (2016.66, 2019) 2.617430830308141

Fragmentation of pp during (1995.01, 2003) 2.6909996627192685
Fragmentation of pp during (2003.01, 2011) 2.4981901856274873
Fragmentation of pp during (2011.01, 2016.4) 3.0232200092366113
Fragmentation of pp during (2016.66, 2019) 2.910780783532844

Fragmentation of pmdb during (1995.01, 2003) 2.786850348165676
Fragmentation of pmdb during (2003.01, 2011) 2.6857991766873774
Fragmentation of pmdb during (2011.01, 2016.4) 2.6721463671924734
Fragmentation of pmdb during (2016.66, 2019) 2.6642746684162524

Fragmentation of pt during (1995.01, 2003) 1.8093114245892123
Fragmentation of pt during (2003.01, 2011) 2.0597650478176948
Fragmentation of pt during (2011.01, 2016.4) 1.9066299336191865
Fragmentation of pt during (2016.66, 2019) 1.93

In [7]:
valid_pps = list(get_valid_pp(nets))

1995.0833333333333 ['pmdb' 'pp' 'dem']
1995.1666666666665 ['pmdb' 'dem' 'pp']
1995.2499999999998 ['pmdb' 'dem' 'pp']
1995.333333333333 ['pmdb' 'dem' 'pp']
1995.4166666666663 ['pmdb' 'dem' 'pp']
1995.4999999999995 ['pmdb' 'dem' 'pp']
1995.5833333333328 ['pmdb' 'pp' 'dem']
1995.666666666666 ['pmdb' 'pp' 'dem']
1995.7499999999993 ['pmdb' 'pp' 'dem']
1995.8333333333326 ['pmdb' 'dem' 'pp']
1995.9166666666658 ['pmdb' 'dem' 'psdb']
1995.999999999999 ['pmdb' 'dem' 'pp']
1996.0833333333323 ['pmdb' 'pp' 'dem']
1996.1666666666656 ['pmdb' 'dem' 'pp']
1996.2499999999989 ['pmdb' 'dem' 'pp']
1996.3333333333321 ['pmdb' 'dem' 'pp']
1996.4166666666654 ['dem' 'pmdb' 'pp']
1996.4999999999986 ['dem' 'pmdb' 'pp']
1996.583333333332 ['dem' 'pp' 'pmdb']
1996.6666666666652 ['dem' 'pmdb' 'pp']
1996.7499999999984 ['pmdb' 'dem' 'pp']
1996.8333333333317 ['pmdb' 'pp' 'psdb']
1996.916666666665 ['pmdb' 'pp' 'dem']
1996.9999999999982 ['pmdb' 'dem' 'psdb']
1997.0833333333314 ['dem' 'pmdb' 'pp']
1997.1666666666647 ['dem'

In [84]:
valid_pps

['pmdb', 'dem', 'pt', 'pp', 'psdb', 'pl']